# CNN
- 완전 연결 네트워크의 문제점으로부터 시작
- fully connected layer의 문제점을 해결하기 위한 방안으로 나옴
- 합성곱 계층은 입력 이미지가 커져도 튜닝해야 할 매개변구 개수에 영향을 주지 않음
- 어떠한 이미지에도 차원 수와 상관없이 적용 가능

# 컨볼루션 연산
- 필터 연산 
입력 데이터에 필터를 통한 어떠한 연산을 진행
필터에 대으앟는 원소끼리 곱하고, 그 합을 구함
연산이 완료된 결과 데이터를 특징 맵이라 부름

- 필터
커널이라고도 칭함
흔히 사진 어플에서 사용하는 '이미지 필터'와 비슷한 개념
필터의 사이즈는 '거의 항상 홀수'
짝수이면 패딩이 비대칭이 되어버림
왼쪽, 오른쪽을 다르게 주어야 함
중심위치가 존재, 즉 구별된 하나의 픽셀(중심 픽셀)이 존재

필터의 학습 파라미터 개수는 입력 데이터의 크기와 상관없이 일정 > 과적합 방지

- 일반적으로 합성곱 연산을 한 후의 데이터의 크기는
(n - f + 1) X (n - f + 1)
n = 입력 데이터의 크기
f = 필터(커널)의 크기

# 패딩과 스트라이드
- 필터(커널) 사이즈와 함께 입력 이미지와 출력 이미지의 사이즈를 결정하기 위해 사용
- 사용자가 결정할 수 있음

# 패딩
- 입력 데이터의 주변을 특정 값으로 채우는 기법 > 주로 0으로 많이 채움
(n+2p-f+1) x (n+2p-f+1)
- 'valid 와 'same'
    - valid : padding을 주지 않음
- 'same'
    - 패딩을 주어 입력 이미지의 크기와 연산 후의 이미지 크기를 같게 함
    - 만약, 필터(커널)의 크기가 k이면, 패딩의 크기는 p= 2/k-1(단, stride = 1)

# 스트라이드 
- 필터를 적용하는 간격을 의미

In [30]:
# 저수준 api 사용
import tensorflow as tf

In [31]:
k, d, n = (3, 16, 32)

In [32]:
kenels_size = [k ,k, d, n]
glorot_uni_initializer = tf.initializers.GlorotUniform()

kernels = tf.Variable(glorot_uni_initializer(kenels_size),
                            trainable = True, name = 'filters')
bias = tf.Variable(tf.zeros(shape = [n]), trainable = True, name= 'bias')

In [33]:
@tf.function
def conv_layer(x, kernels, bias, s):
    z = tf.nn.conv2d(x, kernels, strides = [1, s, s, 1], paddind = 'VALID')
    return tf.nn.relu(z + bias)

In [34]:
class SimpleCNN(tf.keras.layers.Layer):
    def __init__(self, num_kernels = 32, kernel_size = (3,3), stride = 1):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.stride = stride

def build(self, input_shape):
    input_channels = input_shape[-1]
    
    kernels_shape = (*self.kernel_size, input_channels, self.num_kernels)
    glorot_init = tf.initializers.GlorotUniform()
    self.kernels =self.add_weight(
        name = 'kernels', shape = kernels_shape, initializer = glorot_init, trainabel = True)
    self.bias = self.add_weight(name = 'bias', shape = (self.num_kernels,),
                                initializer = 'random_normal', trainable =True)
    
def call(self, inputs):
    return conv_layer(input, self.kernels, self.bias, self.stride)


In [35]:
# 고수준 api  사용
from tensorflow.keras.layers import Conv2D

In [36]:
s = 1

In [37]:
conv = Conv2D(filters = n, kernel_size = (k, k), strides = s,
              padding = 'valid', activation='relu')

# 풀링
- 필터(커널) 사이즈 내에서 특정 값을 추출하는 과정

# 맥스 풀링
- 가장 많이 사용됨
- 일반적으로 stride = 2, kernel_size = 2 를 통해 특징맵의 크기를 절반으로 줄이는 역할
- 모델이 물체의 주요한 특징을 학습할 수 있도록 해줌
>> 일정 부분의 최대값만 출력해서 연산한다.

# 평균 풀링
- 필터내의 픽셀값의 평균을 구하는 과정
- 과거에 많이 사용, 맥스풀링과 비슷함

In [38]:
from tensorflow.keras.layers import AvgPool2D, MaxPool2D

In [39]:
k, s = (3, 1)

In [40]:
avg_pool = AvgPool2D(pool_size = k, strides = [s,s], padding = 'valid')
max_pool = MaxPool2D(pool_size = k, strides = [s,s], padding = 'valid')

# 완전 연결 계층
- 입력으로 받은 텐서를 1차원으로 평면화(flatten) 함
- Dense Layer라고도 함

In [41]:
from tensorflow.keras.layers import Dense

In [42]:
output_size = 64

In [43]:
fc = Dense (units = output_size, activation='relu')

# LeNet-5

In [44]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.datasets import mnist

import numpy as np

In [45]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [46]:
x_train, x_test, =x_train[..., np.newaxis], x_test[..., np.newaxis]

In [47]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

'''
(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)
'''

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


'\n(60000, 28, 28, 1)\n(60000,)\n(10000, 28, 28, 1)\n(10000,)\n'

In [48]:
print(x_train[0 ,:,:, 0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

In [49]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [50]:
print(x_train[0, :,:,0])

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

In [51]:
num_classes = 10
epochs = 100
batch_size = 32

In [54]:
class LeNet5(Model):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = Conv2D(6, kernel_size = (5,5), padding = 'same', activation = 'relu')
        self.conv2 = Conv2D(16, kernel_size = (5,5), activation = 'relu')
        self.max_pool = MaxPool2D(pool_size = (2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation = 'relu')
        self.dense2 = Dense(84, activation = 'relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
def call(self, input_data):
    x = self.max_pool(self.conv1(input_data))
    x = self.max_pool(self.conv2(x))
    x = self.flatten(x)
    x = self.dense3(self.dense2(self.dense1(x)))
    
    return x
        

In [55]:
model = LeNet5(num_classes)

In [59]:
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [60]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience = 3, monitor = 'val_loss'),
             tf.keras.callbacks.TensorBoard(log_dir = './logs', histogram_freq=1)]

In [61]:
model.fit(x_train,y_train,
          batch_size = batch_size,
          epochs = epochs,
          validation_data=(x_test, y_test),
          callbacks = callbacks )

Epoch 1/100


NotImplementedError: in user code:

    File "C:\ProgramData\Anaconda3\envs\tf270gpu\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\envs\tf270gpu\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\envs\tf270gpu\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\envs\tf270gpu\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Anaconda3\envs\tf270gpu\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\envs\tf270gpu\lib\site-packages\keras\engine\training.py", line